# Item 29: Avoid Repeating Work in Comprehensions by Using Assignment Expressions

A common pattern with comprehensions-including `list`, `dict`, and `set` variants-is the need to reference the same computation in multiple places.

In [2]:
# Say we're writing a program to manage orders for a fastener company. As new orders come in from customers, we need
# to be able to tell them whether we can fulfill their orders. We need to verify that a request is sufficiently in
# stock and above the minimum threshold for shipping (in batches of 8)
stock ={
    'nails': 125,
    'screws': 35,
    'wingnuts': 8,
    'washers': 24
}

order = ['screws', 'wingnuts', 'clips']

def get_batches(count, size):
    return count // size

result = {}
for name in order:
    count = stock.get(name, 0)
    batches = get_batches(count, 8)
    if batches:
        result[name] = batches
    
print(result)

{'screws': 4, 'wingnuts': 1}


In [6]:
# Here we implement the above looping logic more succintly using a dictionary comprehension
found = {name: get_batches(stock.get(name, 0), 8)
         for name in order
         if get_batches(stock.get(name, 0), 8)}
print(found)

{'screws': 4, 'wingnuts': 1}


Although the above code is more compact, the problem is that the `get_batches` expression is repeated. This hurts readiblity by adding visual noise that's technically unnecessary. It also increases the likelihood of introducing a bug if the two expressions aren't kept in sync.

In [7]:
# For example, here we changed the first get_batches call to have 4 as its second parameter instead od 8, which
# causes the results to be different
has_bug = {name: get_batches(stock.get(name, 0), 4)
         for name in order
         if get_batches(stock.get(name, 0), 8)}

print('Expected', found)
print('Found', has_bug)

Expected {'screws': 4, 'wingnuts': 1}
Found {'screws': 8, 'wingnuts': 2}


In [8]:
# A solution to these problems is to use the walrus operator (:=) to form an assignment expression as part of the
# comprehension
found = {name: batches for name in order if (batches := get_batches(stock.get(name, 0), 8))}

The assignment expression in the previous code allows us to look up the value for each `order` key in the `stock` dictionary a single time, call `get_batches` once, and then store its corresponding value in the `batches` variable. We can then reference that variable elsewhere in the comprehension to construct the `dict`'s content instead of having to call `get_batches` a second time. 

In [9]:
# Its valid syntax to define an assigment expression in the value expression for a comprehension. But if we try to
# reference the variable it defines in other parts of the comprehension, we might get an exception at runtime because
# of the order in which comprehensions are evaluated
result = {name: (tenth := count //10) for name, count in stock.items() if tenth > 0}

NameError: name 'tenth' is not defined

In [11]:
# We can fix this by moving the assignment expression into the condition and then referencing the variable name it
# defined in the comprehension's value expression
result = {name: tenth for name, count in stock.items() if (tenth := count // 10) > 0}
print(result)

{'nails': 12, 'screws': 3, 'washers': 2}


In [12]:
# If a comprehension uses the walrus operator in the value part of the comprehension and doesn't have a condition, 
# it'll leak the loop variable into the containing scope
half = [(last := count // 2) for count in stock.values()]
print(f'Last item of {half} {last}')

Last item of [62, 17, 4, 12] 12


In [13]:
# This leakage of the loop variable is similar to what happens with a normal for loop
for count in stock.values(): # Leaks loop variable
    pass
print(f'Last item of {list(stock.values())} {count}')

Last item of [125, 35, 8, 24] 24


In [16]:
# However, similar leakage doesn't happen for the loop variables from comprehensions
half = [count // 2 for count in stock.values()]
print(half) # Works
print(count) # Exception because loop variable didn't leak

[62, 17, 4, 12]
24


### It's better not leak loop variables, so the author recommends using assignment expressions only in the condition part of aa comprehension

In [17]:
# Using an assigment expression also works the same way in generator expressions. Here we create an iterator of pairs
# containing the item name and the current count in stock instead of a dict instance
found = ((name, batches) for name in order
         if (batches := get_batches(stock.get(name, 0), 8)))
print(next(found))
print(next(found))

('screws', 4)
('wingnuts', 1)
